In [1]:
import pandas as pd
import numpy as np
import random
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader 

from tqdm.auto import tqdm
from tqdm import trange 

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42) # Seed 고정

In [3]:
hidden_size = 128
num_layers = 12
num_epochs = 50
sequence_length = 64 
batch_size = 512
learning_rate = 0.001
weight_decay = 0.0001
train_val_ratio = 0.9
T_max=100
eta_min=0.001


In [4]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')
info_df = pd.read_csv('./data/building_info.csv')
sample_submission = pd.read_csv('./data/sample_submission.csv')

## preprocessing Data

In [5]:
#  df.loc['B']
build_map_dictionary = {}
for i in range(100):
     build_map_dictionary.update({
         i+1 : info_df['건물유형'][i]
     })
area_warm_dictionary = {}
for i in range(100):
     area_warm_dictionary.update({
         i+1 : info_df['연면적(m2)'][i]
     })
area_cool_map_dictionary = {}
for i in range(100):
     area_cool_map_dictionary.update({
         i+1 : info_df['냉방면적(m2)'][i]
     })


In [6]:
train_df['건물유형'] = train_df['건물번호'].map(build_map_dictionary) 
train_df['연면적(m2)'] = train_df['건물번호'].map(area_warm_dictionary) 
train_df['냉방면적(m2)'] = train_df['건물번호'].map(area_cool_map_dictionary) 

In [7]:
train_df['강수량(mm)'].fillna(0.0, inplace=True)
train_df['풍속(m/s)'].fillna(round(train_df['풍속(m/s)'].mean(),2), inplace=True)
train_df['습도(%)'].fillna(round(train_df['습도(%)'].mean(),2), inplace=True)

In [8]:
train_df.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh),건물유형,연면적(m2),냉방면적(m2)
0,1_20220601 00,1,20220601 00,18.6,0.0,0.9,42.0,NaN,NaN,1085.28,건물기타,110634.0,39570.0
1,1_20220601 01,1,20220601 01,18.0,0.0,1.1,45.0,NaN,NaN,1047.36,건물기타,110634.0,39570.0
2,1_20220601 02,1,20220601 02,17.7,0.0,1.5,45.0,NaN,NaN,974.88,건물기타,110634.0,39570.0
3,1_20220601 03,1,20220601 03,16.7,0.0,1.4,48.0,NaN,NaN,953.76,건물기타,110634.0,39570.0
4,1_20220601 04,1,20220601 04,18.4,0.0,2.8,43.0,NaN,NaN,986.40,건물기타,110634.0,39570.0


In [9]:
# train_df['month'] = train_df['일시'].apply(lambda x : float(x[4:6]))
# train_df['day'] = train_df['일시'].apply(lambda x : float(x[6:8]))
# train_df['time'] = train_df['일시'].apply(lambda x : float(x[9:11]))
# train_df = train_df[train_df.columns[:9].to_list() + train_df.columns[10:].to_list() + train_df.columns[9:10].to_list()]

In [10]:
build_type_map = {}
for idx, type in enumerate(train_df['건물유형'].unique()):
    build_type_map.update({
        type : idx
    })
train_df['building'] = train_df['건물유형'].map(build_type_map)


In [11]:
building_df = pd.get_dummies(train_df['building'],prefix='building')
train_df = pd.concat([building_df, train_df], axis = 1)
y_df = train_df['전력소비량(kWh)']
train_df=train_df.drop(['건물유형','building','일조(hr)','일사(MJ/m2)','건물번호','num_date_time','일시','전력소비량(kWh)'],axis=1)


In [12]:

converted_train_df = train_df[train_df.columns[:16].to_list() + train_df.columns[17:].to_list() + train_df.columns[16:17].to_list()]
# converted_train_df
# converted_train_df


In [13]:
y_df

0         1085.28
1         1047.36
2          974.88
3          953.76
4          986.40
           ...   
203995     881.04
203996     798.96
203997     825.12
203998     640.08
203999     540.24
Name: 전력소비량(kWh), Length: 204000, dtype: float64

## Data loader

In [14]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length, label):
        self.data = data
        self.sequence_length = sequence_length
        self.label = label
#         print(np.shape(self.data))
#         print(f'self.data.shape : {self.data.shape[1]}')
    def __len__(self):
        return len(self.data) - self.sequence_length

    def __getitem__(self, idx):
        if self.label is not None:
            x = torch.tensor(self.data[idx:idx+self.sequence_length, :-1], dtype=torch.float)
            y = torch.tensor(self.label[idx+self.sequence_length], dtype=torch.float)
        else:
            x = torch.tensor(self.data[idx:idx+self.sequence_length], dtype=torch.float)
            y = None            
        return x, y

def create_data_loader(df, sequence_length, batch_size, label=None,  shuffle=True):
    dataset = TimeSeriesDataset(df, sequence_length, label=label)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return data_loader

In [15]:

scaler = MinMaxScaler()
train_data = scaler.fit_transform(converted_train_df.values)

split = int(converted_train_df.shape[0]*train_val_ratio)
y = y_df.to_numpy()
train_data = train_data[:split]
train_y_data = y[:split]

valid_data = train_data[split:]
valid_y_data = y[split:]

# valid_loader = create_data_loader(valid_data, sequence_length, batch_size, shuffle=False)

In [16]:
y

array([1085.28, 1047.36,  974.88, ...,  825.12,  640.08,  540.24])

In [17]:
train_loader = create_data_loader(train_data, sequence_length, batch_size, label=y)
valid_loader = create_data_loader(valid_data, sequence_length, batch_size ,label=y, shuffle=False)


## model

In [18]:
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, sequence_length, num_layers, device):
        super(BiLSTM, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(hidden_size * sequence_length * 2, hidden_size * sequence_length)
        self.dropout1 = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(hidden_size * sequence_length, 1)
        

    def forward(self, x):
#         h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(self.device)
#         c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(self.device)
        h0 = torch.zeros(self.num_layers*2, x.size()[0], self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers*2, x.size()[0], self.hidden_size).to(self.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out.reshape(out.shape[0], -1) 
        out = self.fc1(out)
        out = self.dropout1(out)
        out = self.fc2(out)
        
        return out

In [19]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, sequence_length, num_layers, device):
        super(LSTM, self).__init__()
        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, 1)
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size()[0], self.hidden_size).to(self.device)
        out, _ = self.lstm(x, (h0, c0))
#         print(out.size())
        
        out = out.reshape(out.shape[0], -1) 
        out = self.fc(out)
        return out

In [20]:

#TODO transformer

In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"current device: {device}")
data, label= next(iter(train_loader))
# def __init__(self, input_size, hidden_size, sequence_length, num_layers, device):
model = BiLSTM(input_size = data.size()[2], hidden_size=hidden_size, num_layers=num_layers ,sequence_length=sequence_length, device=device).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,  T_max=T_max, eta_min=eta_min)


current device: cuda


## training

In [ ]:
loss_list = []
min_mean_loss = 10000000
pbar = trange(num_epochs)
for epoch in pbar:
    running_loss = 0.0
    running_mse_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.unsqueeze(1).to(device)
        outputs = model(inputs)
        print(outputs)
        loss = criterion(outputs, labels)
        RMSE_loss = torch.sqrt(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += RMSE_loss.item()
        running_mse_loss += loss.item()
    mean_loss = running_loss/len(train_loader)
#     print(f'mean_loss : {mean_loss}')
    mean_mse_loss = running_mse_loss/len(train_loader)
    loss_list.append(mean_loss)
    scheduler.step()
    pbar.set_postfix({
        'epoch':epoch+1,
        'loss': mean_loss,
        'mse_loss' : mean_mse_loss
    })
    if min_mean_loss > mean_loss:
        torch.save(model.state_dict(), './model_lstm_best.pth')
        min_mean_loss = mean_loss

  0%|                                                                                              | 0/50 [00:00<?, ?it/s]

tensor([[ 2.9456e-02],
        [ 3.8647e-02],
        [ 2.9284e-02],
        [ 2.3155e-02],
        [ 2.9324e-02],
        [ 1.2199e-02],
        [ 3.6733e-02],
        [ 9.8992e-04],
        [ 1.8379e-02],
        [ 3.2721e-02],
        [ 1.5453e-02],
        [ 1.3564e-02],
        [ 2.9138e-02],
        [ 3.0042e-02],
        [ 1.5116e-02],
        [ 2.0119e-02],
        [ 1.6199e-02],
        [ 2.6715e-02],
        [-2.2586e-03],
        [ 6.9707e-03],
        [ 3.7591e-02],
        [ 2.5538e-02],
        [ 3.1312e-02],
        [ 2.3480e-02],
        [ 3.3245e-02],
        [ 3.6609e-02],
        [-9.8870e-04],
        [ 3.1953e-02],
        [ 3.3767e-02],
        [ 3.0450e-02],
        [ 8.5343e-03],
        [ 1.3171e-02],
        [ 1.0119e-02],
        [ 3.6227e-02],
        [ 2.7423e-02],
        [ 3.4017e-02],
        [ 2.0184e-02],
        [ 3.6163e-02],
        [ 2.9307e-02],
        [ 2.5173e-02],
        [-1.8608e-03],
        [ 3.0338e-02],
        [ 2.5238e-02],
        [ 3

tensor([[25.5536],
        [25.5854],
        [25.4988],
        [25.7875],
        [25.9739],
        [25.5053],
        [26.2389],
        [25.3514],
        [25.6852],
        [25.6600],
        [25.8130],
        [25.8250],
        [25.0338],
        [26.1101],
        [26.1733],
        [25.3162],
        [25.0684],
        [25.7210],
        [25.5981],
        [25.7498],
        [25.4847],
        [25.1172],
        [26.1588],
        [25.1663],
        [25.4646],
        [25.0937],
        [25.1631],
        [25.5348],
        [25.3148],
        [25.6833],
        [25.2677],
        [24.9112],
        [25.3464],
        [25.6800],
        [25.0461],
        [25.6870],
        [25.3223],
        [25.2415],
        [25.5464],
        [25.7042],
        [25.6318],
        [25.8730],
        [25.7420],
        [25.8537],
        [25.6607],
        [25.4962],
        [25.6333],
        [25.4072],
        [25.6961],
        [25.5750],
        [25.9881],
        [25.6822],
        [25.

tensor([[78.0662],
        [79.0668],
        [76.8633],
        [77.8477],
        [78.8959],
        [78.3019],
        [79.1168],
        [79.3134],
        [78.7539],
        [77.7877],
        [78.0017],
        [78.2006],
        [76.6226],
        [77.0321],
        [77.1751],
        [76.7455],
        [78.1673],
        [77.6139],
        [77.3982],
        [77.7711],
        [78.7688],
        [78.0027],
        [76.9013],
        [78.8838],
        [77.6999],
        [78.9664],
        [78.2844],
        [77.6304],
        [77.3879],
        [77.3601],
        [77.8332],
        [78.1020],
        [77.5455],
        [76.7381],
        [76.4601],
        [76.1786],
        [77.0331],
        [77.8966],
        [77.5727],
        [77.8159],
        [78.7112],
        [77.3387],
        [77.9630],
        [76.6869],
        [78.2255],
        [78.9445],
        [77.5804],
        [77.3860],
        [77.2455],
        [77.0934],
        [78.9460],
        [79.6302],
        [77.

tensor([[182.4566],
        [188.6804],
        [184.3593],
        [182.9694],
        [182.5240],
        [185.8647],
        [187.5246],
        [181.5678],
        [181.7891],
        [187.2498],
        [183.0368],
        [185.6225],
        [182.8143],
        [181.3189],
        [183.2057],
        [187.4832],
        [184.3777],
        [181.4770],
        [182.2508],
        [183.8665],
        [186.8624],
        [182.1794],
        [184.4272],
        [182.1773],
        [188.5925],
        [185.3347],
        [181.4094],
        [187.9471],
        [181.5748],
        [183.4475],
        [184.8981],
        [179.2176],
        [187.3288],
        [187.4100],
        [181.8096],
        [185.5626],
        [182.4653],
        [181.0549],
        [184.2303],
        [184.7892],
        [187.2835],
        [183.9542],
        [186.4519],
        [182.7054],
        [186.8956],
        [183.9754],
        [183.0989],
        [179.3335],
        [184.2950],
        [182.1370],


...

tensor([[2508.4832],
        [2516.0955],
        [2544.9622],
        [2521.6924],
        [2502.4719],
        [2554.1687],
        [2549.9075],
        [2486.6201],
        [2584.0515],
        [2506.3630],
        [2558.2639],
        [2496.7085],
        [2521.8721],
        [2464.8782],
        [2496.0583],
        [2497.6321],
        [2525.9060],
        [2513.2195],
        [2563.7354],
        [2545.9700],
        [2533.3914],
        [2587.4465],
        [2489.7419],
        [2494.6616],
        [2510.0999],
        [2494.1040],
        [2486.5083],
        [2493.1941],
        [2509.5710],
        [2517.8743],
        [2505.9841],
        [2508.4058],
        [2547.8674],
        [2563.0530],
        [2478.5481],
        [2527.6956],
        [2556.2268],
        [2533.8049],
        [2475.1409],
        [2557.8284],
        [2493.5613],
        [2514.2097],
        [2497.2090],
        [2481.5488],
        [2548.9585],
        [2439.5088],
        [2502.6785],
        [2492

tensor([[2391.0496],
        [2446.9539],
        [2418.9832],
        [2399.6121],
        [2432.6157],
        [2422.5613],
        [2413.9807],
        [2362.9595],
        [2394.0156],
        [2423.2542],
        [2399.2495],
        [2446.0732],
        [2389.9832],
        [2418.1111],
        [2404.2659],
        [2414.1162],
        [2386.5386],
        [2436.6536],
        [2522.3042],
        [2458.3115],
        [2386.0107],
        [2426.3555],
        [2431.7336],
        [2360.6008],
        [2416.6204],
        [2382.1506],
        [2452.1692],
        [2429.0110],
        [2391.3542],
        [2408.8188],
        [2431.7073],
        [2423.7910],
        [2408.3296],
        [2382.5767],
        [2442.1377],
        [2434.1550],
        [2436.8623],
        [2337.5520],
        [2421.1433],
        [2401.4395],
        [2416.3794],
        [2424.7961],
        [2471.5403],
        [2401.0125],
        [2433.2190],
        [2442.3840],
        [2425.4514],
        [2413

tensor([[2351.8044],
        [2330.1365],
        [2330.8792],
        [2374.5344],
        [2296.1465],
        [2341.9236],
        [2293.6016],
        [2337.4788],
        [2382.6250],
        [2302.6001],
        [2318.1250],
        [2369.6018],
        [2342.8840],
        [2285.7585],
        [2356.2883],
        [2406.5652],
        [2350.6653],
        [2351.4788],
        [2354.4226],
        [2372.8516],
        [2415.0723],
        [2340.2690],
        [2317.0325],
        [2357.0325],
        [2320.7561],
        [2348.3555],
        [2324.7117],
        [2395.6284],
        [2392.1609],
        [2360.3625],
        [2339.6687],
        [2302.0737],
        [2337.5095],
        [2334.2603],
        [2365.7664],
        [2361.7058],
        [2395.0164],
        [2310.7898],
        [2283.3232],
        [2294.6001],
        [2378.9819],
        [2357.0342],
        [2311.7825],
        [2336.4258],
        [2372.9431],
        [2242.2102],
        [2248.7898],
        [2353

tensor([[2250.7961],
        [2296.9729],
        [2221.6418],
        [2277.0579],
        [2277.5742],
        [2281.2664],
        [2297.2439],
        [2239.6943],
        [2307.9709],
        [2268.9082],
        [2284.7422],
        [2286.8992],
        [2249.8333],
        [2204.4690],
        [2247.0012],
        [2216.2959],
        [2327.0220],
        [2308.1970],
        [2349.7683],
        [2251.4460],
        [2249.0212],
        [2294.5583],
        [2372.3784],
        [2324.3125],
        [2324.0916],
        [2269.6685],
        [2262.5000],
        [2302.9238],
        [2322.7739],
        [2294.0671],
        [2280.5889],
        [2309.1052],
        [2279.4946],
        [2334.5339],
        [2351.6294],
        [2296.5479],
        [2278.1077],
        [2379.3572],
        [2288.9014],
        [2301.1362],
        [2274.7471],
        [2244.3240],
        [2294.7727],
        [2291.3428],
        [2274.2410],
        [2273.8730],
        [2233.0608],
        [2250

tensor([[2247.1460],
        [2283.9368],
        [2212.1738],
        [2246.8186],
        [2266.4763],
        [2234.8953],
        [2313.1396],
        [2304.6833],
        [2277.4734],
        [2255.1282],
        [2243.7024],
        [2179.2292],
        [2301.1931],
        [2251.0964],
        [2244.5298],
        [2203.7029],
        [2217.2351],
        [2260.0225],
        [2236.3552],
        [2256.4553],
        [2297.2205],
        [2235.6819],
        [2306.4460],
        [2262.5493],
        [2278.4438],
        [2246.7976],
        [2277.7224],
        [2261.6021],
        [2260.7444],
        [2320.9058],
        [2309.6155],
        [2201.7512],
        [2279.2454],
        [2284.7415],
        [2286.1890],
        [2318.7310],
        [2263.4607],
        [2245.6877],
        [2256.6040],
        [2264.6672],
        [2306.0884],
        [2330.2063],
        [2208.4587],
        [2265.8313],
        [2181.0835],
        [2248.8665],
        [2239.8689],
        [2245

In [ ]:
torch.save(model.state_dict(), './model_lstm.pth')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 4))
plt.plot(loss_list, '-*')
plt.title("Training Loss")
plt.xlabel("epoch")
plt.show()

## proprocessing test data

In [ ]:
last_train_data = converted_train_df.loc[204000-24:,:]
# last_train_data

In [ ]:
test_df['습도(%)'] = test_df['습도(%)'].astype('float64')
test_df['month'] = test_df['일시'].apply(lambda x : float(x[4:6]))
test_df['day'] = test_df['일시'].apply(lambda x : float(x[6:8]))
test_df['time'] = test_df['일시'].apply(lambda x : float(x[9:11]))

test_df['건물유형'] = test_df['건물번호'].map(build_map_dictionary) 
test_df['연면적(m2)'] = test_df['건물번호'].map(area_warm_dictionary) 
test_df['냉방면적(m2)'] = test_df['건물번호'].map(area_cool_map_dictionary) 


In [ ]:
build_type_map = {}
for idx, type in enumerate(test_df['건물유형'].unique()):
    build_type_map.update({
        type : idx
    })
test_df['building'] = test_df['건물유형'].map(build_type_map)
building_test_df = pd.get_dummies(test_df['building'],prefix='building')
test_df=test_df.drop(['건물유형','building','건물번호','num_date_time','일시'],axis=1)
converted_test_df = pd.concat([building_test_df, test_df], axis = 1)
converted_test_df = converted_test_df[converted_test_df.columns[:16].to_list() + converted_test_df.columns[19:].to_list() + converted_test_df.columns[16:19].to_list()]


In [ ]:
final_df = pd.concat((converted_test_df, pd.DataFrame(np.zeros(converted_test_df.shape[0]))),axis=1)
final_df = final_df.rename({0:'전력소비량(kWh)'},axis=1)
test_df = pd.concat((last_train_data, final_df)).reset_index(drop=True)


In [ ]:
test_data = scaler.transform(test_df.values) # train과 동일하게 scaling
test_data.shape

In [ ]:
# Dataset & DataLoader
test_dataset = TimeSeriesDataset(test_data, window_size)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
model.load_state_dict(torch.load('./model_bilstm.pth'))


In [ ]:
# test_loader.next.iter()
d,la= next(iter(test_loader))
d

## inference

In [ ]:
model.eval()

test_predictions = []

# with torch.no_grad():
#     for i, inputs in enumerate(test_loader):
#         inputs = inputs.to(device)
#         outputs = model(inputs)
#     for i in range(test_data.shape[0] - window_size):
#         x = torch.Tensor(test_data[i:i+window_size,:]).to(device)
#         new_x = model(x.view(1,window_size,-1))
#         print(outputs.detach().cpu().numpy().item())
#         test_data[i+window_size,-1] = new_x # 입력 업데이트
#         test_predictions.append(outputs.detach().cpu().numpy().item()) # 예측 결과 저장
        
        
        
#  for i, (inputs, labels) in enumerate(train_loader):
#         inputs = inputs.to(device)
#         labels = labels.unsqueeze(1).to(device)
#         outputs = model(inputs)

with torch.no_grad():
    for i in range(test_data.shape[0] - window_size):
        x = torch.Tensor(test_data[i:i+window_size,:]).to(device)
        print(x.view(1,window_size,-1))
        new_x = model(x.view(1,window_size,-1))
        print(new_x)
#         test_data[i+window_size,-1] = new_x # 입력 업데이트
#         test_predictions.append(new_x.detach().cpu().numpy().item()) # 예측 결과 저장

In [ ]:
test_data

In [ ]:
predictions = scaler.inverse_transform(test_data)[24:,-1] # 원래 scale로 복구


In [ ]:
sample_submission['answer'] = predictions
sample_submission.head()

In [ ]:
sample_submission.to_csv('bilstm_baseline_submission.csv', index=False)